In [1]:
import aiohttp
import asyncio
import nest_asyncio
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import json
import os


In [2]:
nest_asyncio.apply()

API_BASE_URL = "https://en.wikipedia.org/w/api.php"
ARTICLES_PER_TOPIC = 6000
MAX_DEPTH = 6
CHECKPOINT_FILE = 'wiki_scraping_checkpoint.json'
ARTICLES_BEFORE_CHECKPOINT = 1000


In [3]:
# Topics and corresponding subtopics
topics_dictionary = {
"Health": [
    "Healthcare Systems", "Public Health", "Mental Health", "Nutrition", 
    "Diseases", "Health Technology", "Medicine", "Surgery", "Vaccination", 
    "Epidemiology", "Health Policy", "Health Insurance", "Fitness", "Wellness", 
    "Preventive Care", "Pediatrics", "Geriatrics", "Women's Health", "Men's Health", 
    "Teen Health", "Sexual Health", "Reproductive Health", "Maternal Health", 
    "Global Health", "Health Education", "Health Research", "Clinical Trials", 
    "Medical Ethics", "Health Economics", "Pharmaceuticals", "Biotechnology", 
    "Alternative Medicine", "Psychotherapy", "Physical Therapy", "Occupational Therapy", 
    "Dental Health", "Vision Care", "Hearing Health", "Orthopedics", "Oncology", 
    "Cardiology", "Neurology", "Gastroenterology", "Endocrinology", "Dermatology", 
    "Allergies", "Infectious Diseases", "Chronic Diseases", "Rare Diseases", 
    "Mental Disorders", "Sleep Disorders", "Addiction", "Pain Management", 
    "Aging", "Childhood Obesity", "Diabetes", "Cancer", "HIV/AIDS", "Tuberculosis", 
    "Malaria", "Zika Virus", "COVID-19", "Heart Disease", "Stroke", "Healthcare Inequality", 
    "Telemedicine", "Remote Health Monitoring", "Health Data Analytics", "Genomics", 
    "Personalized Medicine", "Health Apps", "Wearable Health Devices", "Mental Health Apps", 
    "Health Blogs", "Health Podcasts", "Medical News", "Health and Safety Regulations", 
    "Emergency Preparedness", "First Aid", "Medical Training", "Health Fairs", 
    "Health Campaigns", "Health Awareness", "Antibiotic Resistance", "Vaccine Research", 
    "Disease Eradication", "Health Conferences", "Health Benefits", "Health Risks", 
    "Health Advice", "Health Trends", "Health Statistics", "Health Reports", 
    "Health Systems Analysis", "Health Professional Shortage", "Health Budgets", 
    "Health Legislation", "Health Reforms", "Health Technology Assessment", "Health Interventions"
],
"Environment": [
    "Climate Change", "Global Warming", "Renewable Energy", "Solar Power", "Wind Energy",
    "Hydro Power", "Bioenergy", "Geothermal Energy", "Energy Efficiency", "Carbon Footprint",
    "Sustainability", "Sustainable Development", "Environmental Impact", "Pollution", "Air Quality",
    "Water Quality", "Soil Contamination", "Recycling", "Waste Management", "Composting",
    "Landfills", "Deforestation", "Reforestation", "Forest Management", "Wildlife Conservation",
    "Endangered Species", "Biodiversity", "Habitat Destruction", "Ocean Conservation", "Coral Reefs",
    "Marine Pollution", "Fishing", "Aquaculture", "Overfishing", "Conservation Efforts",
    "National Parks", "Protected Areas", "Eco-Tourism", "Green Cities", "Urban Planning",
    "Green Building", "Smart Cities", "Public Transport", "Electric Vehicles", "Sustainable Agriculture",
    "Organic Farming", "Pesticides", "Soil Health", "Crop Rotation", "Food Security",
    "Water Conservation", "Water Treatment", "Desalination", "Irrigation", "Rainwater Harvesting",
    "Climate Resilience", "Adaptation Strategies", "Mitigation Strategies", "Environmental Policies",
    "Environmental Law", "Environmental Education", "Eco-Friendly Products", "Green Technology",
    "Green Businesses", "Corporate Sustainability", "Environmental Ethics", "Environmental Activism",
    "Grassroots Movements", "NGOs", "Environmental Justice", "Community Engagement", "Public Awareness",
    "Environmental Conferences", "Environmental Research", "Climate Models", "Weather Patterns",
    "Natural Disasters", "Disaster Response", "Ecosystem Services", "Land Degradation", "Desertification",
    "Glaciers", "Polar Regions", "Sea Level Rise", "Thermal Expansion", "Climate Negotiations",
    "International Agreements", "Emissions Trading", "Carbon Markets", "Carbon Sequestration",
    "Greenhouse Gases", "Atmospheric Sciences", "Meteorology", "Environmental Monitoring", "Remote Sensing",
    "GIS", "Satellite Imagery", "Drone Technology", "Environmental Engineering", "Renewable Energy Certificates"
]
,
"Technology": [
    "Innovation", "Emerging Technologies", "Artificial Intelligence", "Machine Learning",
    "Deep Learning", "Neural Networks", "Robotics", "Automation", "Natural Language Processing",
    "Computer Vision", "Data Science", "Big Data", "Analytics", "Cloud Computing",
    "Edge Computing", "Quantum Computing", "Cybersecurity", "Information Security", "Blockchain",
    "Cryptocurrencies", "Bitcoin", "Ethereum", "Smart Contracts", "Fintech", "Insurtech",
    "Regtech", "Virtual Reality", "Augmented Reality", "Mixed Reality", "Gaming",
    "Esports", "Mobile Applications", "Software Development", "App Development", "UI/UX Design",
    "Web Development", "Responsive Design", "Digital Marketing", "SEO", "Social Media",
    "E-Commerce", "Online Shopping", "Streaming Services", "Music Streaming", "Video Streaming",
    "Drones", "Wearable Technology", "Fitness Trackers", "Smart Watches", "Health Tech",
    "Medtech", "Biotechnology", "Pharmatech", "Educational Technology", "Edtech",
    "Learning Management Systems", "Virtual Classrooms", "E-Learning", "MOOCs", "Remote Work",
    "Telecommuting", "Collaboration Tools", "Project Management Software", "CRM Software",
    "ERP Systems", "IoT", "Smart Home Devices", "Smart Cities", "Connected Cars",
    "Autonomous Vehicles", "Electric Vehicles", "Battery Technology", "Energy Storage",
    "Renewable Energy Tech", "Solar Panels", "Wind Turbines", "Tech Startups", "Venture Capital",
    "Silicon Valley", "Tech Hubs", "Digital Transformation", "Agile Methodologies", "Scrum",
    "Digital Ethics", "Privacy", "Data Protection", "GDPR", "Regulatory Compliance", 
    "Tech Policy", "International Trade in Tech", "Tech Conferences", "Tech Exhibitions",
    "Product Development", "User-Centered Design", "Human-Computer Interaction", "Tech Philanthropy"
],
"Economy": [
    "Global Economy", "Economic Growth", "Recession", "Inflation", "Deflation",
    "Monetary Policy", "Fiscal Policy", "Central Banking", "Interest Rates", "Exchange Rates",
    "Foreign Exchange", "Financial Markets", "Stock Market", "Bond Market", "Commodity Market",
    "Trading", "Investments", "Mutual Funds", "Hedge Funds", "Private Equity",
    "Venture Capital", "Startups", "Business Cycles", "Supply Chain", "Logistics",
    "Consumer Behavior", "Retail Industry", "E-Commerce", "Real Estate", "Housing Market",
    "Insurance", "Banking", "Commercial Banking", "Corporate Finance", "Public Finance",
    "Taxation", "Budgeting", "Government Spending", "Public Debt", "Economic Policy",
    "Economic Indicators", "Labor Market", "Unemployment", "Employment Statistics", "Job Market",
    "Wage Growth", "Productivity", "Human Capital", "Economic Development", "Emerging Markets",
    "International Trade", "Export-Import", "Trade Agreements", "Trade Barriers", "Economic Sanctions",
    "Economic Integration", "Economic Forecasting", "Economic Analysis", "Economic Research", "Economic Theories",
    "Microeconomics", "Macroeconomics", "Behavioral Economics", "Development Economics", "Health Economics",
    "Environmental Economics", "Energy Economics", "Resource Economics", "Agricultural Economics", "Industrial Organization",
    "Market Regulation", "Antitrust Laws", "Economic Crises", "Financial Crises", "Economic Recovery",
    "Economic Sustainability", "Corporate Governance", "Business Ethics", "Corporate Social Responsibility", "Innovation Economics",
    "Technology and Economy", "Digital Economy", "Crypto Economy", "Blockchain in Economy", "Financial Technology",
    "Mobile Banking", "Online Banking", "Credit", "Debt", "Savings",
    "Pensions", "Retirement Planning", "Wealth Management", "Economic Inequality", "Poverty Reduction",
    "Social Welfare", "Universal Basic Income", "Economic Mobility", "Trade Wars", "Economic Diplomacy"
],
"Entertainment": [
    "Cinema", "Theater", "Television", "Streaming Services", "Music Industry",
    "Film Production", "Documentaries", "Reality TV", "Talk Shows", "Comedy",
    "Drama", "Action Films", "Horror Films", "Musicals", "Opera",
    "Ballet", "Dance", "Visual Arts", "Painting", "Sculpture",
    "Photography", "Art Exhibitions", "Museums", "Galleries", "Book Publishing",
    "Literature", "Novels", "Poetry", "Biographies", "Essays",
    "Journalism", "News Media", "Magazines", "Radio", "Podcasts",
    "Video Games", "Esports", "Game Development", "VR Gaming", "Mobile Gaming",
    "Concerts", "Live Performances", "Festivals", "Award Shows", "Celebrity Culture",
    "Fashion", "Red Carpet Events", "Entertainment Marketing", "Media Promotion", "Public Relations",
    "Animation", "Graphic Novels", "Comics", "Manga", "Anime",
    "Fan Culture", "Social Media Trends", "Influencer Marketing", "Viral Content", "Content Creation",
    "Digital Art", "Web Series", "Music Videos", "Choreography", "Talent Shows",
    "Art Criticism", "Entertainment Law", "Copyright Issues", "Licensing", "Entertainment Technology",
    "Sound Engineering", "Film Editing", "Screenwriting", "Directing", "Producing",
    "Casting", "Cinematography", "Costume Design", "Set Design", "Lighting Design",
    "Special Effects", "Animation Technology", "3D Modeling", "Virtual Sets", "Streaming Technology"
],
"Sports": [
    "Olympics", "Paralympics", "World Cup", "NBA", "NFL",
    "MLB", "NHL", "Soccer", "Basketball", "Baseball",
    "Hockey", "Tennis", "Golf", "Cricket", "Rugby",
    "Boxing", "MMA", "Athletics", "Track and Field", "Swimming",
    "Cycling", "Skiing", "Snowboarding", "Surfing", "Motorsports",
    "Formula One", "Rally Racing", "Motorbike Racing", "Equestrian", "Polo",
    "Gymnastics", "Fencing", "Archery", "Shooting Sports", "Wrestling",
    "Judo", "Taekwondo", "Karate", "Sailing", "Rowing",
    "Canoeing", "Kayaking", "Diving", "Water Polo", "Triathlon",
    "Marathon", "Ultramarathon", "Adventure Racing", "Obstacle Racing", "Fitness",
    "Bodybuilding", "Weightlifting", "Yoga", "Pilates", "Aerobics",
    "Sports Medicine", "Sports Nutrition", "Sports Psychology", "Coaching", "Sports Training",
    "Sports Gear", "Sports Apparel", "Sports Technology", "Sports Analytics", "Sports Statistics",
    "Sports Management", "Sports Marketing", "Sports Sponsorship", "Sports Media", "Sports Broadcasting",
    "Sports Betting", "Fantasy Sports", "Esports", "Sports Law", "Sports Ethics",
    "Sports Governance", "International Sports Federations", "Sports Events", "Sports Venues", "Sports Tourism",
    "Sports History", "Sports Culture", "Sports Fan Engagement", "Youth Sports", "Community Sports"
],
"Politics": [
    "Political Science", "Political Theory", "Democracy", "Authoritarianism", "Communism",
    "Socialism", "Capitalism", "Anarchism", "Liberalism", "Conservatism",
    "Nationalism", "Populism", "Political Ideologies", "Political Systems", "Governance",
    "Government Institutions", "Parliaments", "Senates", "Congress", "Presidential Systems",
    "Parliamentary Systems", "Federalism", "State Governments", "Local Governments", "Elections",
    "Voting", "Electoral Systems", "Political Campaigns", "Political Parties", "Political Debates",
    "Political Communication", "Political Rhetoric", "Media and Politics", "Political Journalism", "Public Opinion",
    "Polls", "Political Activism", "Protests", "Rallies", "Civil Rights",
    "Human Rights", "Freedom of Speech", "Freedom of the Press", "Judicial Systems", "Constitutional Law",
    "Legal Rights", "Legislation", "Bills", "Laws", "Regulations",
    "Public Policy", "Health Policy", "Education Policy", "Foreign Policy", "Defense Policy",
    "Environmental Policy", "Economic Policy", "Trade Policy", "Immigration Policy", "Labor Policy",
    "Welfare Policy", "Tax Policy", "Security Policy", "Diplomacy", "International Relations",
    "Global Governance", "Treaties", "International Organizations", "UN", "NATO",
    "EU", "ASEAN", "African Union", "Geopolitics", "International Conflicts",
    "War", "Peacekeeping", "Conflict Resolution", "Terrorism", "Counterterrorism",
    "Political Corruption", "Ethics in Politics", "Transparency", "Accountability", "Political Leadership",
    "Political Philosophy", "Political History", "Political Biographies", "Political Theories", "Political Analysis",
    "Policy Analysis", "Policy Making", "Public Administration", "Civic Engagement", "Voter Turnout",
    "Political Education", "Political Sociology", "Political Economy", "Crisis Management", "Political Crises"
],
"Education": [
    "Early Childhood Education", "Primary Education", "Secondary Education", "Higher Education", "Adult Education",
    "Vocational Training", "Professional Development", "Continuing Education", "Online Learning", "Distance Learning",
    "E-learning", "Blended Learning", "Educational Technology", "MOOCs", "Virtual Classrooms",
    "Learning Management Systems", "Educational Apps", "Digital Classrooms", "STEM Education", "Arts Education",
    "Literacy", "Numeracy", "Reading Skills", "Writing Skills", "Mathematics Education",
    "Science Education", "History Education", "Geography Education", "Language Learning", "Second Language Acquisition",
    "Special Education", "Gifted Education", "Inclusive Education", "Learning Disabilities", "Educational Psychology",
    "Pedagogy", "Teaching Methods", "Classroom Management", "Curriculum Development", "Lesson Planning",
    "Assessment Methods", "Standardized Testing", "Educational Standards", "Educational Policy", "Education Reform",
    "School Governance", "School Funding", "Education Law", "Teacher Training", "Teacher Certification",
    "Faculty Development", "Professional Ethics", "Educational Leadership", "School Administration", "School Safety",
    "Bullying Prevention", "Student Counseling", "Career Counseling", "Academic Advising", "Student Life",
    "Extracurricular Activities", "Sports in Education", "Arts in Education", "Civic Education", "Environmental Education",
    "Health Education", "Nutrition Education", "Sex Education", "Drug and Alcohol Education", "Parental Involvement",
    "Community Involvement", "Educational Grants", "Scholarships", "Student Loans", "Education Technology Innovation",
    "Educational Research", "Educational Conferences", "Educational Publishing", "Textbook Publishing", "Educational Media"
]
,
"Travel": [
    "Adventure Travel", "Leisure Travel", "Business Travel", "Ecotourism", "Sustainable Travel",
    "Cultural Tourism", "Heritage Tourism", "Wildlife Tourism", "Sports Tourism", "Medical Tourism",
    "Wellness Tourism", "Religious Tourism", "Gastronomic Tourism", "Wine Tourism", "Agritourism",
    "Rural Tourism", "Urban Tourism", "Luxury Travel", "Budget Travel", "Solo Travel",
    "Group Travel", "Family Travel", "Senior Travel", "Student Travel", "Accessible Travel",
    "Cruise Travel", "Rail Travel", "Road Trips", "Backpacking", "Hiking",
    "Camping", "Skiing", "Beach Vacations", "Island Getaways", "Resorts",
    "Spas", "World Heritage Sites", "Landmarks", "Monuments", "Museums",
    "Galleries", "Theater Trips", "Music Festivals", "Film Festivals", "Conferences",
    "Trade Shows", "Travel Safety", "Travel Health", "Travel Insurance", "Travel Documentation",
    "Visas", "Passports", "Customs", "Border Security", "Airlines",
    "Airports", "Flight Booking", "Hotel Booking", "Accommodation", "Hospitality",
    "Travel Guides", "Travel Agencies", "Tour Operators", "Tour Guides", "Travel Apps",
    "Travel Blogs", "Travel Vlogs", "Travel Photography", "Travel Writing", "Travel Journalism",
    "Cultural Exchange", "Globalization and Travel", "Economic Impact of Travel", "Environmental Impact of Travel", "Travel Trends",
    "Travel Planning", "Trip Itinerary Planning", "Destination Marketing", "Tourism Development", "Tourism Marketing",
    "Tourist Behavior", "Travel Technology", "Virtual Tourism", "Space Tourism", "Pandemic Impact on Travel"
]
,
"Food": [
    "Culinary Arts", "Cooking", "Baking", "Grilling", "Roasting",
    "Frying", "Steaming", "Poaching", "Simmering", "Sous-vide",
    "Food Preparation", "Food Presentation", "Food Pairing", "Food Tasting", "Food Criticism",
    "Food Safety", "Food Hygiene", "Food Storage", "Food Preservation", "Canning",
    "Pickling", "Fermentation", "Drying", "Smoking", "Curing",
    "Food Processing", "Food Manufacturing", "Food Packaging", "Food Labeling", "Food Distribution",
    "Food Retail", "Supermarkets", "Grocery Stores", "Farmers' Markets", "Organic Food",
    "Local Food", "Sustainable Food", "Veganism", "Vegetarianism", "Plant-based Diets",
    "Meat", "Poultry", "Fish", "Seafood", "Dairy Products",
    "Eggs", "Cheese", "Fruits", "Vegetables", "Grains",
    "Legumes", "Nuts", "Seeds", "Spices", "Herbs",
    "Beverages", "Coffee", "Tea", "Wine", "Beer",
    "Spirits", "Cocktails", "Juices", "Smoothies", "Water",
    "Soft Drinks", "Desserts", "Snacks", "Confectionery", "Chocolates",
    "Cakes", "Cookies", "Pastries", "Breads", "Pizzas",
    "Pasta", "Rice Dishes", "Soups", "Stews", "Salads",
    "Sandwiches", "Burgers", "Food Trends", "Dietary Trends", "Nutrition",
    "Dietary Supplements", "Food Allergies", "Gluten-Free", "Keto Diet", "Paleo Diet",
    "Food Security", "Food Policy", "Food Education", "Food Science", "Food Technology",
    "Food Ethics", "Food History", "Culinary Schools", "Chef Training", "Food Shows"
]
}


In [4]:
def load_checkpoint():
    """Load the checkpoint data from a file."""
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, 'r') as file:
            data = json.load(file)
            return data['articles'], set(data['urls'])
    return {}, set()

def save_checkpoint(documented_articles, unique_article_urls):
    """Save the checkpoint data to a file."""
    checkpoint_data = {
        'articles': documented_articles,
        'urls': list(unique_article_urls)
    }
    with open(CHECKPOINT_FILE, 'w') as file:
        json.dump(checkpoint_data, file, indent=4)

async def retrieve_articles(session, search_topic, continue_params=None):
    search_params = {
        "action": "query",
        "list": "search",
        "srsearch": search_topic,
        "format": "json",
        "srlimit": 200
    }
    if continue_params:
        search_params.update(continue_params)

    async with session.get(API_BASE_URL, params=search_params) as response:
        response_data = await response.json()
        search_continue = response_data.get('continue')
        articles = response_data["query"]["search"] if 'query' in response_data else []
        return articles, search_continue

async def retrieve_revision_id(session, article_title):
    revision_params = {
        "action": "query",
        "prop": "revisions",
        "titles": article_title,
        "rvprop": "ids",
        "format": "json"
    }
    async with session.get(API_BASE_URL, params=revision_params) as response:
        response_data = await response.json()
        page_data = next(iter(response_data["query"]["pages"].values()))
        return page_data["revisions"][0]["revid"]

async def extract_article_summary(session, url):
    async with session.get(url) as response:
        page_content = BeautifulSoup(await response.text(), 'html.parser')
        return ' '.join(p.text for p in page_content.select('p'))

async def explore_deep_links(session, url, depth, visited_urls, articles_accumulator, documented_articles, unique_article_urls, main_topic):
    if depth > MAX_DEPTH or len(articles_accumulator) >= ARTICLES_PER_TOPIC:
        return
    async with session.get(url) as response:
        soup = BeautifulSoup(await response.text(), 'html.parser')
        for link in soup.find_all('a', href=True):
            link_url = urljoin(url, link['href'])
            if link_url not in visited_urls and '/wiki/' in urlparse(link_url).path and not any(x in link_url for x in [':', '#', 'Main_Page']):
                visited_urls.add(link_url)
                summary = await extract_article_summary(session, link_url)
                if summary and len(summary) >= 250:
                    article_title = link_url.split('/')[-1].replace('_', ' ')
                    revision_id = await retrieve_revision_id(session, article_title)
                    articles_accumulator.append({
                        'revision_id': revision_id,
                        'title': article_title,
                        'summary': summary,
                        'url': link_url,
                        'topic': main_topic
                    })
                    if len(articles_accumulator) % ARTICLES_BEFORE_CHECKPOINT == 0:
                        save_checkpoint(documented_articles, unique_article_urls)
                    if len(articles_accumulator) >= ARTICLES_PER_TOPIC:
                        break
                await explore_deep_links(session, link_url, depth + 1, visited_urls, articles_accumulator, documented_articles, unique_article_urls, main_topic)

async def collect_articles_for_subtopic(session, main_topic, subtopic, articles_accumulator, unique_article_urls, documented_articles):
    continue_params = None
    while True:
        articles_found, continue_params = await retrieve_articles(session, subtopic, continue_params)
        for article in articles_found:
            article_url = f"https://en.wikipedia.org/wiki/{article['title'].replace(' ', '_')}"
            if article_url in unique_article_urls or len(articles_accumulator) >= ARTICLES_PER_TOPIC:
                continue
            unique_article_urls.add(article_url)
            summary = await extract_article_summary(session, article_url)
            if summary and len(summary) >= 250:
                revision_id = await retrieve_revision_id(session, article['title'])
                articles_accumulator.append({
                    'revision_id': revision_id,
                    'title': article['title'],
                    'summary': summary,
                    'url': article_url,
                    'topic': main_topic
                })
                if len(articles_accumulator) % ARTICLES_BEFORE_CHECKPOINT == 0:
                    save_checkpoint(documented_articles, unique_article_urls)
        if not continue_params or len(articles_accumulator) >= ARTICLES_PER_TOPIC:
            break

async def execute_scraping():
    async with aiohttp.ClientSession() as session:
        documented_articles, unique_article_urls = load_checkpoint()
        collection_tasks = []
        for topic, subtopics in topics_dictionary.items():
            documented_articles.setdefault(topic, [])
            for subtopic in subtopics:
                task = asyncio.create_task(
                    collect_articles_for_subtopic(
                        session, topic, subtopic, documented_articles[topic], unique_article_urls, documented_articles
                    )
                )
                collection_tasks.append(task)
                if len(documented_articles[topic]) >= ARTICLES_PER_TOPIC:
                    break
        await asyncio.gather(*collection_tasks)
        save_checkpoint(documented_articles, unique_article_urls)

        with open('wiki_scraped_articles_final.json', 'w') as file:
            json.dump(documented_articles, file, indent=4)


In [6]:
if __name__ == "__main__":
    asyncio.run(execute_scraping())

In [7]:
def load_data(filepath):
    with open(filepath, 'r') as file_handle:
        return json.load(file_handle)

def present_documents(data):
    for topic, documents in data.items():
        print(f"\nTopic: {topic}, Number of Documents: {len(documents)}\n")
        for doc in documents[:10]:
            title = doc.get('title', 'Title Unavailable')
            url = doc.get('url', 'URL Unavailable')
            summary = doc.get('summary', 'Summary Unavailable')[:200]
            print(f"Title: {title}\nURL: {url}\nSummary: {summary}\n---")

data_path = 'wiki_scraped_articles_final.json'
scraped_info = load_data(data_path)
present_documents(scraped_info)


Topic: Health, Number of Documents: 6099

Title: Health insurance
URL: https://en.wikipedia.org/wiki/Health_insurance
Summary: 
 Health insurance or medical insurance (also known as medical aid in South Africa) is a type of insurance that covers the whole or a part of the risk of a person incurring medical expenses. As with o
---
Title: Health technology
URL: https://en.wikipedia.org/wiki/Health_technology
Summary: Health technology is defined by the World Health Organization as the "application of organized knowledge and skills in the form of devices, medicines, vaccines, procedures, and systems developed to so
---
Title: Sexual and reproductive health
URL: https://en.wikipedia.org/wiki/Sexual_and_reproductive_health
Summary: 
 Sexual and reproductive health (SRH)[1] is a field of research, health care, and social activism that explores the health of an individual's reproductive system and sexual well-being during all stag
---
Title: Biotechnology
URL: https://en.wikipedia.org/wiki/B